In [7]:
import pandas as pd
import numpy as np
import os
import re

In [3]:
df1 = pd.read_csv(r".\MEL.csv") 
df2 = pd.read_csv(r".\shipsEar_prompts_4.csv")  

df2['Filename'] = df2['Filename'].str.replace('.wav', '', regex=False)


In [4]:
merged_df = pd.merge(df1, df2[['Filename', 'prompt_en']], 
                     on='Filename', 
                     how='left')

merged_df.to_csv('zero-shot.csv', index=False)
print("合并完成！")
print(f"合并后的数据形状: {merged_df.shape}")
print("前5行数据:")
print(merged_df.head())

合并完成！
合并后的数据形状: (11210, 6)
前5行数据:
   ID                       Filename                  SegmentedFilename  \
0   6  6__10_07_13_marDeCangas_Entra    6__10_07_13_marDeCangas_Entra_1   
1   6  6__10_07_13_marDeCangas_Entra   6__10_07_13_marDeCangas_Entra_10   
2   6  6__10_07_13_marDeCangas_Entra  6__10_07_13_marDeCangas_Entra_100   
3   6  6__10_07_13_marDeCangas_Entra  6__10_07_13_marDeCangas_Entra_101   
4   6  6__10_07_13_marDeCangas_Entra  6__10_07_13_marDeCangas_Entra_102   

         Type  ClassID                                          prompt_en  
0  Passengers        2  The sound belongs to a Passengers (size class ...  
1  Passengers        2  The sound belongs to a Passengers (size class ...  
2  Passengers        2  The sound belongs to a Passengers (size class ...  
3  Passengers        2  The sound belongs to a Passengers (size class ...  
4  Passengers        2  The sound belongs to a Passengers (size class ...  
